In [175]:
import numpy as np
import pandas as pd

In [176]:
#Convert json to pandas frame for easier manipulation. Sort user_id and time_played values to get songs played ordered in time:
song = pd.read_json('song.json').sort_values(['user_id', 'time_played'])[['user_id','song_played']]
song.head(10)

,user_id,song_played
1952,1,Yesterday
619,1,While My Guitar Gently Weeps
2719,1,The Long And Winding Road
669,1,Reprise / Day in the Life
353,1,I Feel Fine
993,1,Hello Goodbye
627,1,Here Comes The Sun
1373,1,Can't Buy Me Love
1563,1,Birthday
3172,1,Here Comes The Sun


In [183]:
# To get next song played from song played already arranged in time order, shift the song_played columns up by 1 row:
song['next_song'] = song.groupby(['user_id'])['song_played'].shift(-1)
song.head(3)

,user_id,song_played,next_song
1952,1,Yesterday,While My Guitar Gently Weeps
619,1,While My Guitar Gently Weeps,The Long And Winding Road
2719,1,The Long And Winding Road,Reprise / Day in the Life


In [185]:
# Find the value counts of next song by each song_played group:
song_1 = song.groupby(['song_played'])['next_song'].agg(['value_counts']).reset_index()
song_1.head(3)

,song_played,next_song,value_counts
0,A Day In The Life,Come Together,16
1,A Day In The Life,Revolution,15
2,A Day In The Life,Get Back,12


In [187]:
# The most likely song to be played next is the song with highest value_counts:
idx = song_1.groupby(['song_played'])['value_counts'].transform(max) == song_1['value_counts']
song_2 = song_1[idx][['song_played', 'next_song']]
song_2.head(3)

,song_played,next_song
0,A Day In The Life,Come Together
43,A Hard Day's Night,Let It Be
55,A Saturday Club Xmas/Crimble Medley,A Day In The Life


In [196]:
# Set song_played as index for easier searching:
song_3 = song_2.set_index('song_played')
song_3.head(8)

,next_song
song_played,
A Day In The Life,Come Together
A Hard Day's Night,Let It Be
A Saturday Club Xmas/Crimble Medley,A Day In The Life
A Saturday Club Xmas/Crimble Medley,GIRL
ANYTIME AT ALL,Can't Buy Me Love
ANYTIME AT ALL,Helter Skelter
ANYTIME AT ALL,Here Comes The Sun
ANYTIME AT ALL,IN MY LIFE


In [192]:
# Define the function to get next song to be played. The function takes two arguments: given playing song, and the dataframe I have created above. In many cases, there are several songs
# likely to be played after a given song, I choose the first alphabetically:
def get_next_songs(song_played, song_3):
    next = song_3[song_3.index == song_played]['next_song'].tolist()[0]
    return songs

In [193]:
get_next_songs('Eight Days A Week', song_3)

'Come Together'

In [194]:
get_next_songs('ANYTIME AT ALL', song_3)

"Can't Buy Me Love"